## Load libraries, Set Filepaths

In [20]:
%cd '~/matt/courses/deeplearning1/nbs'
import os
from utils import *
import keras.layers.convolutional as convolutional
from keras.models import Sequential
import allnew_vgg
reload(allnew_vgg)
import allnew_vgg as av
reload(allnew_vgg)

/home/ubuntu/matt/courses/deeplearning1/nbs


<module 'allnew_vgg' from 'allnew_vgg.pyc'>

In [21]:
%cd '~/matt/courses/deeplearning1/nbs'
current_dir = os.getcwd()
data_dir = current_dir + '/data/kag_dogscats/' #sample/
# data_dir = current_dir + '/data/kag_dogscats/sample/'
results_path=data_dir + 'results/'
train_path=data_dir + 'train/'
valid_path=data_dir + 'valid/'
test_path=data_dir + 'test/'
model_path=data_dir + 'models/'
print current_dir, data_dir

/home/ubuntu/matt/courses/deeplearning1/nbs
/home/ubuntu/matt/courses/deeplearning1/nbs /home/ubuntu/matt/courses/deeplearning1/nbs/data/kag_dogscats/


In [4]:
## build python functions to check for data and grab using cli otherwise
PROJECT_DIR=current_dir
DATA_HOME_DIR=data_dir
#Create directories
%mkdir -p $DATA_HOME_DIR
%cd $DATA_HOME_DIR
%mkdir -p valid
%mkdir -p results
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p test/unknown

/home/ubuntu/matt/courses/deeplearning1/nbs/data/kag_dogscats


## Set variables

In [22]:
batch_size = 64
no_of_epochs = 8

## Build VGG model and load pretrained weights. Then Finetune

In [23]:
vgg = av.preloaded_vgg()

In [24]:
vgg.mnn_finetuner()

starting number of layers:  38
ending number of layers:  38


In [25]:
conv_layers, fc_layers = vgg.layer_divider()

## Take convolutional layers and build new model class to work off of

In [45]:
mgg = av.adjusted_vgg()

In [46]:
mgg.create_conv_model(conv_layers)

## Gather Batches

In [28]:
train = mgg.mnn_batches(train_path, batch_size=batch_size, train=True, shuffle=False)
valid = mgg.mnn_batches(valid_path, batch_size=batch_size*2, train=False, shuffle=False)

train_classes = train.classes
valid_classes = valid.classes
train_labels = onehot(train_classes)
valid_labels = onehot(valid_classes)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


## Train weights for base model built above, save them to iterate new layers

In [40]:
train_features = mgg.model.predict_generator(generator=train, val_samples=train.nb_sample)
valid_features = mgg.model.predict_generator(generator=valid, val_samples=valid.nb_sample)

In [41]:
save_array(model_path+'train_convlayer_features.bc', train_features)
save_array(model_path+'valid_convlayer_features.bc', valid_features)

## Load base weights

In [29]:
train_features = load_array(model_path+'train_convlayer_features.bc')
valid_features = load_array(model_path+'valid_convlayer_features.bc')

## Create a model of just dense layers, using shape of trained features above

In [47]:
mgg.create_fc_model(train_features, p=.4)

In [48]:
mgg.add_fc_weights(fc_layers)

In [49]:
mgg.fc_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
# fc_model = mgg.dense_layer_model(fc_layers, input_shape=train_features.shape[1:])

In [50]:
mgg.fc_model.fit(x=train_features, y=train_labels, nb_epoch=8, batch_size=64, validation_data=(valid_features, valid_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/8
23000/23000 [==============================] - 38s - loss: 0.3053 - acc: 0.9546 - val_loss: 0.0476 - val_acc: 0.9840
Epoch 2/8
23000/23000 [==============================] - 39s - loss: 0.0747 - acc: 0.9757 - val_loss: 0.0459 - val_acc: 0.9860
Epoch 3/8
23000/23000 [==============================] - 41s - loss: 0.0316 - acc: 0.9895 - val_loss: 0.0807 - val_acc: 0.9775
Epoch 4/8
23000/23000 [==============================] - 40s - loss: 0.0324 - acc: 0.9902 - val_loss: 0.0591 - val_acc: 0.9810
Epoch 5/8
23000/23000 [==============================] - 41s - loss: 0.0218 - acc: 0.9927 - val_loss: 0.1213 - val_acc: 0.9825
Epoch 6/8
23000/23000 [==============================] - 40s - loss: 0.0229 - acc: 0.9930 - val_loss: 0.1016 - val_acc: 0.9805
Epoch 7/8
23000/23000 [==============================] - 40s - loss: 0.0242 - acc: 0.9930 - val_loss: 0.0812 - val_acc: 0.9840
Epoch 8/8
23000/23000 [==============================] - 41s -

In [52]:
weights_name = 'dogscats_dropout_iter.h5'
mgg.fc_model.save_weights(results_path+weights_name)
# fc_model.load_weights(results_path+'dogcats_clean.h5')

## Combine Weights from New Model with Old Model

In [159]:
for layer in fc_layers: 
    mgg.model.add(layer)

for l1,l2 in zip(fc_model.layers, mgg.model.layers[len(conv_layers):]):
    l2.set_weights(l1.get_weights())

## Generate Test Batches and Make Predictions

In [161]:
test_batches = get_batches(test_path, shuffle=False, batch_size=batch_size*2, class_mode=None)

Found 12500 images belonging to 1 classes.


In [163]:
predictions = mgg.model.predict_generator(test_batches, test_batches.nb_sample)

In [165]:
filenames = test_batches.filenames

save_array(results_path + 'test_predictions.dat', predictions)
save_array(results_path + 'filenames.dat', filenames)

## Formatting submission correctly, including "clipped" max and min

In [169]:
subm = mgg.submission_formatter(filenames, predictions, 1, .055, .945)

## Saving Predictions CSV and Generating a Click-able Link 

In [170]:
np.savetxt(results_path+'clean_model_submission.csv', subm, fmt='%d,%.5f', header='id,label', comments='')

In [186]:
# if filelink is not working, use commands below in command line
# %kg submit results/clean_model_submission.csv -m "Adding dropout and augmentation to VGG16."

ERROR:root:Line magic function `%kg` not found.


/home/ubuntu/courses/deeplearning1/nbs/results


In [183]:
# from IPython.display import FileLink
# FileLink(results_path+'clean_model_submission.csv')

/home/ubuntu/courses/deeplearning1/nbs/data/kag_dogscats/results/clean_model_submission.csv